### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

### Открываем файл purchases

In [2]:
%%time
t1 = time.clock()
path_to_data = '/home/anatoliy_cherepanov/X5retail/RetailHero-data'
full_df = pd.read_csv(path_to_data+'/purchases.csv')

CPU times: user 1min 19s, sys: 15.7 s, total: 1min 34s
Wall time: 1min 34s


In [3]:
full_df.head()

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN


# Agregate numeric features

### Объявляем колонки для агрегаций

In [4]:
# По клиентам
col_aggr_by_clients = ['regular_points_received', 'express_points_received',
                       'regular_points_spent', 'express_points_spent', 'purchase_sum', 'product_quantity',
                       'trn_sum_from_iss', 'trn_sum_from_red']

# По клиентам и транзакиям
col_aggr_by_clients_trans = ['product_quantity']

### Агрегирующие функции

In [5]:
name_functions = ['count', 'max', 'min', 'mean', 'median', 'std', 'sum']
def stat_function(data, name_function):
    
    if name_function=='count':
        return_data = data.count()
    elif name_function=='max':
        return_data = data.max()
    elif name_function=='min':
        return_data = data.min()
    elif name_function=='mean':
        return_data = data.mean()
    elif name_function=='median':
        return_data = data.median()
    elif name_function=='std':
        return_data = data.std()
    elif name_function=='sum':
        return_data = data.sum()
        
    return return_data
        

### Агрегация

In [6]:
%%time

# Инициализируем
df_features = pd.DataFrame()

# По клиентам
for column in col_aggr_by_clients:
    for function in name_functions[1:]:
        df_features[function+'_'+column] = stat_function(full_df.groupby(['client_id'], sort=False)[column], function)
        
# По клиентам и транзакиям
for column in col_aggr_by_clients_trans:
    for function in name_functions[1:]:
        data = pd.DataFrame(full_df.groupby(['client_id','transaction_id'], sort=False)[column].sum())
        df_features[function+'_'+column] = stat_function(data.groupby(['client_id'], sort=False)[column], function)

# Число позиций 
for function in name_functions[1:]:
    data = pd.DataFrame(full_df.groupby(['client_id','transaction_id'], sort=False).count())['transaction_datetime']
    df_features[function+'_positions'] = stat_function(data.groupby(['client_id'], sort=False),function)
    
# Число посещений
data = pd.DataFrame(full_df.groupby(['client_id','transaction_id'], sort=False)['transaction_id'].count())
df_features['num_visits'] = data.groupby(['client_id'], sort=False).count()

del data

CPU times: user 4min 13s, sys: 30 s, total: 4min 43s
Wall time: 4min 43s


### Промежуточный результат

In [7]:
df_features.head()

,max_regular_points_received,min_regular_points_received,mean_regular_points_received,median_regular_points_received,std_regular_points_received,sum_regular_points_received,max_express_points_received,min_express_points_received,mean_express_points_received,median_express_points_received,...,median_trn_sum_from_red,std_trn_sum_from_red,sum_trn_sum_from_red,max_positions,min_positions,mean_positions,median_positions,std_positions,sum_positions,num_visits
client_id,,,,,,,,,,,,,,,,,,,,,
000012768d,10.0,2.0,7.551923,8.0,2.581039,392.7,0.0,0.0,0.000000,0.0,...,NaN,NaN,NaN,19,6,13.000000,13.5,5.715476,52,4
000036f903,7.0,0.2,2.150000,1.6,1.689583,348.3,60.0,0.0,2.222222,0.0,...,NaN,NaN,NaN,9,1,5.062500,5.0,2.108929,162,32
000048b7a6,11.2,0.5,3.907143,2.3,3.449472,218.8,0.0,0.0,0.000000,0.0,...,NaN,NaN,NaN,10,2,7.000000,8.0,2.777460,56,8
000073194a,17.2,0.1,6.326829,4.9,5.369094,518.8,0.0,0.0,0.000000,0.0,...,20.5,8.084965,131.0,11,1,4.823529,3.0,3.321056,82,17
00007c7133,10.4,1.5,5.538554,7.2,3.569376,459.7,0.0,0.0,0.000000,0.0,...,55.0,78.007326,552.0,12,1,7.545455,8.0,2.733629,83,11


# Agregate Datetime features

In [8]:
full_df.head()

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN


### Создаем датафрейм клиент - время транзакции

In [9]:
%%time

# Клиенты
set_clients = set(full_df.client_id)

# Кусок из полной таблицы
client_date_df = full_df[['client_id','transaction_datetime']]

# Уникальные пары
client_date = sorted(set([(x,y) for x,y in zip(client_date_df['client_id'], client_date_df['transaction_datetime'])]))

# Датафрейм уникальных пар
client_date_df = pd.DataFrame(list(client_date),columns=['client_id','date'])
client_date_df.head(10)

CPU times: user 47.1 s, sys: 1.3 s, total: 48.4 s
Wall time: 48.3 s


### Создаем словарь клиент - дни

In [10]:
%%time

client_dates = dict()
for i in range(len(client_date_df)):
    client = client_date_df['client_id'][i]
    date = client_date_df['date'][i]
    if client not in client_dates.keys():
        client_dates[client]=[]
    client_dates[client].append(date)

### Агрегирующая datetime функция

In [11]:
def create_feat_from_days(client_id, dict_client_date):
    '''
    Uses list with dates of visits.
    Returns list with delays between visits in days.
    Also returns number of visits grouped by weekday. 

    '''
    
    def str_to_datetime(date):
        time = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        return time
    
    # Получаем лист посещений
    client_visit_days = dict_client_date[client_id]
    n_visits = len(client_visit_days)
    
    #Число посещений утром, днем, вечером, ночью
    part_of_day = [0,0,0,0]
    
        
    def part_day(hour):
        if (3<hour<=9):
            part = 0
        elif (9<hour<=15):
            part = 1
        elif (15<hour<=21):
            part = 2
        elif (hour>21 or hour<=3):
            part = 3
            
        return part
            
    for date in client_visit_days:
        hour = str_to_datetime(date).hour
        part_of_day[part_day(hour)]+=1
    
    #Продолжительность посещений
    if len(client_visit_days)==1:

        # Если у клиента только одно посещение помечаем его 1
        timespan = 1
    else:
        timespan = 1 + (str_to_datetime(client_visit_days[-1])-str_to_datetime(client_visit_days[0])).days 
    
    week_time = 7
    month_time = 30
    quartal_time = 90
    year_time = 365
    
    #Число посещений в неделю
    n_week = n_visits/timespan/week_time
    
    
    # Число посещений по дням недели
    weekday_list = [0,0,0,0,0,0,0]
    for date in client_visit_days:
        n_week_day = str_to_datetime(date).weekday()
        weekday_list[n_week_day]+=1

    list_delay_days = []
    if len(client_visit_days)==1:

        # Если у клиента только одно посещение помечаем его -1
        list_delay_days.append(-1)
    else:
        # Вычисляем дилэй между ближайшими посещениями
        for i in range(len(client_visit_days)-1):
            date_0 = datetime.strptime(client_visit_days[i], '%Y-%m-%d %H:%M:%S')
            date_1 = datetime.strptime(client_visit_days[i+1], '%Y-%m-%d %H:%M:%S')
            delay = (date_1 - date_0).days

            list_delay_days.append(delay)

            
            
            
    list_of_functions = [np.max, np.min, np.mean, np.median, np.std]
    name_of_functions = ['max', 'min', 'mean', 'meadian', 'std']
    # Составляем финальный лист фичей
    result_list = []
    columns = []
    for func, name in zip(list_of_functions, name_of_functions):
        result_list.append(func(list_delay_days))
#         columns.append(name+'_delay')
    
    result_list.extend(part_of_day)
#     columns.extend(['morning','midday', 'evening', 'night'])
    
    result_list.append(timespan)
#     columns.append('timespan')
    
    result_list.append(n_week)
#     columns.append('vis_p_week')
    
    result_list.extend(weekday_list)
#     columns.extend(['weekday_1', 
#                     'weekday_2',
#                     'weekday_3',
#                     'weekday_4',
#                     'weekday_5',
#                     'weekday_6',
#                     'weekday_7',])
    
    return result_list

columns = ['max_delay', 'min_delay', 'mean_delay', 'median_delay', 'std_delay',
            'morning', 'midday', 'evening', 'night',
            'timespan', 'vis_p_week',
            'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']

### Применяем агрегацию по всем клиентам

In [12]:
%%time
date_df = []
keys = []
for client in client_dates.keys():
    date_df.append(create_feat_from_days(client, client_dates))
    keys.append(client)
    
date_df = pd.DataFrame(date_df, index=keys, columns=columns)  
date_df.head()

CPU times: user 7min 16s, sys: 803 ms, total: 7min 17s
Wall time: 7min 17s


### Объединяем два датафрейма

In [13]:
%%time
t2 = time.clock()
final_df = df_features.join(date_df)

CPU times: user 302 ms, sys: 81 µs, total: 302 ms
Wall time: 299 ms


# Результат

In [14]:
print(f'Время прогона = {(t2-t1)/60} мин')

Время прогона = 19.975261 мин


In [15]:
final_df.head()

,max_regular_points_received,min_regular_points_received,mean_regular_points_received,median_regular_points_received,std_regular_points_received,sum_regular_points_received,max_express_points_received,min_express_points_received,mean_express_points_received,median_express_points_received,...,night,timespan,vis_p_week,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
client_id,,,,,,,,,,,,,,,,,,,,,
000012768d,10.0,2.0,7.551923,8.0,2.581039,392.7,0.0,0.0,0.000000,0.0,...,0,104,0.005495,0,0,0,1,1,1,1
000036f903,7.0,0.2,2.150000,1.6,1.689583,348.3,60.0,0.0,2.222222,0.0,...,0,109,0.041940,3,7,6,3,3,1,9
000048b7a6,11.2,0.5,3.907143,2.3,3.449472,218.8,0.0,0.0,0.000000,0.0,...,0,87,0.013136,1,2,1,1,0,2,1
000073194a,17.2,0.1,6.326829,4.9,5.369094,518.8,0.0,0.0,0.000000,0.0,...,0,111,0.021879,2,0,1,4,3,5,2
00007c7133,10.4,1.5,5.538554,7.2,3.569376,459.7,0.0,0.0,0.000000,0.0,...,0,101,0.015559,2,1,1,1,1,2,3
